In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Iterable, Optional

# ---------------------------
# Utilidades de detección
# ---------------------------
_COMMON_TIME_COLS = ("fecha", "time", "datetime", "date", "timestamp")

def _guess_time_col(df: pd.DataFrame, preferred: Optional[str] = None) -> str:
    if preferred and preferred in df.columns:
        return preferred
    # Buscar por nombre común (insensible a mayúsculas)
    lower_map = {c.lower(): c for c in df.columns}
    for k in _COMMON_TIME_COLS:
        if k in lower_map:
            return lower_map[k]
    # Si no hay, intentar detectar columna datetime por dtype
    dt_candidates = [c for c in df.columns if np.issubdtype(df[c].dtype, np.datetime64)]
    if dt_candidates:
        return dt_candidates[0]
    raise ValueError(
        "No se encontró columna de tiempo. Específica 'time_col' y/o convierte tu columna a datetime."
    )

def _to_datetime(series: pd.Series) -> pd.Series:
    # Convierte robustamente, tolera varios formatos y NaT en errores
    return pd.to_datetime(series, errors="coerce", infer_datetime_format=True)

# ---------------------------
# Carga y limpieza
# ---------------------------
def cargar_csv(ruta: str | Path, time_col: Optional[str] = None, sep: Optional[str] = None) -> pd.DataFrame:
    """
    Carga CSV y convierte la columna de tiempo a datetime si existe.
    - sep: si None, pandas intenta inferir; pon "," o ";" según tu archivo.
    """
    df = pd.read_csv(ruta, sep=sep)
    # Estandarizar nombres quitando espacios extra
    df.columns = [c.strip() for c in df.columns]
    # Intentar convertir columna de tiempo
    try:
        col_t = _guess_time_col(df, preferred=time_col)
        df[col_t] = _to_datetime(df[col_t])
    except ValueError:
        pass  # No hay columna temporal; el resto del flujo sigue funcionando
    return df

def limpiar_basico(
    df: pd.DataFrame,
    drop_duplicates: bool = True,
    sort_by_time: bool = True,
    time_col: Optional[str] = None,
) -> pd.DataFrame:
    df2 = df.copy()
    if drop_duplicates:
        df2 = df2.drop_duplicates()
    # Orden temporal si aplica
    try:
        tcol = _guess_time_col(df2, preferred=time_col)
        if sort_by_time and tcol in df2.columns:
            df2 = df2.sort_values(tcol)
    except ValueError:
        pass
    return df2.reset_index(drop=True)

# ---------------------------
# Reportes rápidos
# ---------------------------
def reporte_info(df: pd.DataFrame) -> None:
    print("=== Forma del DataFrame ===")
    print(df.shape)
    print("\n=== Tipos de datos ===")
    print(df.dtypes)
    print("\n=== % de valores faltantes por columna ===")
    print((df.isna().mean() * 100).round(2).sort_values(ascending=False))
    print("\n=== Estadísticos descriptivos (numéricos) ===")
    print(df.select_dtypes(include=np.number).describe().T)

# ---------------------------
# Outliers por IQR
# ---------------------------
def winsorizar_iqr(df: pd.DataFrame, cols: Optional[Iterable[str]] = None, factor: float = 1.5) -> pd.DataFrame:
    """
    Limita (cap) valores fuera de [Q1 - factor*IQR, Q3 + factor*IQR].
    No elimina filas; acota valores extremos. Útil para sensores o mediciones erráticas.
    """
    df2 = df.copy()
    if cols is None:
        cols = df2.select_dtypes(include=np.number).columns
    for c in cols:
        s = df2[c].dropna()
        if s.empty:
            continue
        q1, q3 = s.quantile([0.25, 0.75])
        iqr = q3 - q1
        low, high = q1 - factor * iqr, q3 + factor * iqr
        df2[c] = df2[c].clip(lower=low, upper=high)
    return df2

# ---------------------------
# Relleno de faltantes (series de tiempo)
# ---------------------------
def rellenar_ts(
    df: pd.DataFrame,
    method: str = "ffill",
    time_col: Optional[str] = None,
    limit: Optional[int] = None,
) -> pd.DataFrame:
    """
    Rellena valores faltantes con forward fill (ffill), backward fill (bfill) o interpolación lineal ('interpolate').
    """
    df2 = df.copy()
    try:
        tcol = _guess_time_col(df2, preferred=time_col)
        df2 = df2.sort_values(tcol)
    except ValueError:
        tcol = None

    if method in ("ffill", "bfill"):
        df2 = df2.fillna(method=method, limit=limit)
    elif method == "interpolate":
        df2 = df2.interpolate(method="time" if tcol else "linear", limit=limit)
    else:
        raise ValueError("Método no soportado. Usa 'ffill', 'bfill' o 'interpolate'.")
    return df2

# ---------------------------
# Resampleo temporal
# ---------------------------
def resamplear(
    df: pd.DataFrame,
    rule: str = "D",            # 'H' por hora, 'D' diario, 'W' semanal, 'M' mensual
    agg: str | dict = "mean",
    time_col: Optional[str] = None,
) -> pd.DataFrame:
    tcol = _guess_time_col(df, preferred=time_col)
    if not np.issubdtype(df[tcol].dtype, np.datetime64):
        raise ValueError(f"La columna de tiempo '{tcol}' no es datetime. Conviértela primero.")
    df2 = df.set_index(tcol).sort_index()
    return df2.resample(rule).agg(agg).reset_index()

# ---------------------------
# Estadísticas por grupo (ej. por 'Season' o 'Area')
# ---------------------------
def estadisticas_por_grupo(
    df: pd.DataFrame,
    by: Iterable[str] | str,
    value: Optional[str] = None,
    funcs: Iterable = ("count", "mean", "std", "min", "median", "max")
) -> pd.DataFrame:
    """
    - by: columna(s) categóricas (ej. 'Season', 'Area').
    - value: si None, aplica a todas las numéricas; si string, solo esa variable.
    """
    if value:
        g = df.groupby(by)[value].agg(funcs)
    else:
        g = df.groupby(by).agg({c: funcs for c in df.select_dtypes(include=np.number).columns})
    return g

# ---------------------------
# Correlaciones
# ---------------------------
def matriz_correlacion(df: pd.DataFrame) -> pd.DataFrame:
    return df.select_dtypes(include=np.number).corr()

# ---------------------------
# Gráficas rápidas
# ---------------------------
def plot_ts(
    df: pd.DataFrame,
    y_cols: Iterable[str],
    time_col: Optional[str] = None,
    title: Optional[str] = None,
    marker: str = "o",
    alpha: float = 0.9,
    connect: bool = True,
    figsize=(10, 5),
) -> None:
    """
    Grafica una o varias columnas vs tiempo. No fija colores manualmente.
    """
    tcol = _guess_time_col(df, preferred=time_col)
    if not np.issubdtype(df[tcol].dtype, np.datetime64):
        raise ValueError(f"La columna de tiempo '{tcol}' no es datetime. Conviértela primero.")
    pdf = df.sort_values(tcol)

    plt.figure(figsize=figsize)
    for col in y_cols:
        if connect:
            plt.plot(pdf[tcol], pdf[col], label=col, marker=marker, linestyle="-", alpha=alpha)
        else:
            plt.scatter(pdf[tcol], pdf[col], label=col, alpha=alpha)
    plt.title(title or f"{', '.join(y_cols)} vs tiempo")
    plt.xlabel("Tiempo")
    plt.ylabel("Valor")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_rolling_mean(
    df: pd.DataFrame,
    y_col: str,
    window: int = 7,
    time_col: Optional[str] = None,
    title: Optional[str] = None,
) -> None:
    """
    Muestra la serie original y su media móvil (suavizado) para detectar tendencia.
    """
    tcol = _guess_time_col(df, preferred=time_col)
    pdf = df.sort_values(tcol).copy()
    pdf["rolling"] = pdf[y_col].rolling(window=window, min_periods=max(1, window//2)).mean()

    plt.figure(figsize=(10,5))
    plt.plot(pdf[tcol], pdf[y_col], label=y_col, marker="o", linestyle="-", alpha=0.5)
    plt.plot(pdf[tcol], pdf["rolling"], label=f"Media móvil ({window})", linestyle="-")
    plt.title(title or f"{y_col}: original vs media móvil")
    plt.xlabel("Tiempo")
    plt.ylabel(y_col)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# ---------------------------
# Ejemplo de uso (modifica rutas/columnas y ejecuta este archivo)
# ---------------------------
if __name__ == "__main__":
    # === 1) CONFIGURA ESTAS VARIABLES ===
    RUTA = "datos.csv"          # <-- tu CSV
    TIME_COL = None             # p.ej., "fecha" si quieres forzarlo. Deja None para autodetectar.
    COLS_Y = ["temperatura"]    # <-- variables a graficar vs tiempo (pueden ser varias)
    GRUPO = "Season"            # <-- usa la que tengas (ej. 'Season', 'Area', etc.). Si no existe, ignora esta parte.

    # === 2) CARGA Y LIMPIEZA ===
    df = cargar_csv(RUTA, time_col=TIME_COL)
    df = limpiar_basico(df)

    print("\n--- REPORTE INICIAL ---")
    reporte_info(df)

    # === 3) TRATAMIENTO DE OUTLIERS (WINSORIZAR) ===
    df_w = winsorizar_iqr(df, factor=1.5)
    print("\nSe aplicó winsorización IQR a columnas numéricas (factor=1.5).")

    # === 4) RELLENO (opcional) ===
    df_f = rellenar_ts(df_w, method="interpolate")  # "ffill", "bfill" o "interpolate"
    print("Se aplicó relleno de faltantes (interpolate).")

    # === 5) RESAMPLEO (opcional) ===
    try:
        df_d = resamplear(df_f, rule="D", agg="mean")  # diario
        print("\n--- RESUMEN DIARIO (primeras filas) ---")
        print(df_d.head())
    except Exception as e:
        print(f"\n(No se resampleó: {e})")
        df_d = df_f

    # === 6) ESTADÍSTICAS POR GRUPO (opcional) ===
    if isinstance(GRUPO, str) and GRUPO in df_d.columns:
        print(f"\n--- ESTADÍSTICAS POR GRUPO: {GRUPO} ---")
        print(estadisticas_por_grupo(df_d, by=GRUPO).round(3).head())

    # === 7) CORRELACIONES ===
    print("\n--- MATRIZ DE CORRELACIÓN ---")
    print(matriz_correlacion(df_d).round(3))

    # === 8) GRÁFICAS ===
    try:
        plot_ts(df_d, y_cols=COLS_Y, time_col=TIME_COL, title="Serie(s) vs tiempo")
        # Media móvil para la primera variable
        if COLS_Y:
            plot_rolling_mean(df_d, y_col=COLS_Y[0], window=7, time_col=TIME_COL)
    except Exception as e:
        print(f"(No se pudo graficar: {e})")

    # === 9) GUARDAR LIMPIO (opcional) ===
    salida = Path(RUTA).with_name(Path(RUTA).stem + "_limpio.csv")
    df_d.to_csv(salida, index=False)
    print(f"\n✔ Archivo limpio guardado en: {salida}")
